In [ ]:

import ctypes
import os
from abc import ABC, abstractmethod


In [ ]:
path = os.path.join(os.getcwd(), "cpp/interface.so")
dll = ctypes.CDLL(path)

getBoard=dll._Z10base_statei
copyBoard=dll._Z9copyBoardP6EBoard
updateState=dll._Z10next_stateP6EBoardl
getAction=dll._Z10getActionsP6EBoardPl
actionToString=dll._Z14actionToStringl
printAct=dll._Z16printActionFancyl

actionToString.restype = ctypes.c_char_p
actionToString.argtypes = [ctypes.c_uint64]
printAct.argtypes = [ctypes.c_uint64]
getAction.argtypes = [ctypes.c_void_p,ctypes.POINTER(ctypes.c_uint64)]

In [ ]:
b=getBoard()

In [ ]:
RisType = ctypes.c_uint64 * 256

ris=RisType()
ris[0]=2

In [ ]:
import time
time.time()

In [ ]:
from game_rule import DLLGameRule 
GR=DLLGameRule()
import time
import random
import ctypes
import math
import time

In [ ]:
totMov = 0
t_start = time.time()

for rt in range(200):
    state = GR.init_state()
    i = 0

    ris = (ctypes.c_int * 256)()  # buffer for actions
    while i<200000:

        ris=GR.getActions(state)
        # .EBoard_getNextsActions(board, ris)

        num_actions = ris[0]
        m = random.randint(1, num_actions)
        GR.next_state(state, ris[m])

        out = GR.checkStatus(state)
        if out != 1:
            print(i)
            break
        totMov += 1
        
        i += 1

    
t_end = time.time()
print("Tempo totale mossa", (t_end - t_start) /totMov, "s")
print(totMov)


In [ ]:
state = GR.init_state()
ris=GR.getActions(state)
GR.next_state(state, ris[0])
out = GR.checkStatus(state)
        

In [ ]:

ris=GR.getActions(state)
GR.next_state(state, ris[0])
out = GR.checkStatus(state)

In [ ]:
out

In [ ]:
for i in range(1,ris[0]+1):
    print(ris[i], actionToString(ris[i]).decode('utf-8'))
    printAct(ris[i])
    print("")

In [ ]:
path = os.path.join(os.getcwd(), "cpp/engine.dll")
dll = ctypes.CDLL(path)
        

In [ ]:
dll._Z10validMovesv.restype=ctypes.POINTER(ctypes.POINTER(ctypes.c_char))# ctypes.POINTER(ctypes.c_char_p)
r=dll._Z10validMovesv()


In [ ]:
r[2][5]

In [1]:
class game_rule_cpp_imp(game_rule):
    def __init__(self):
        path = os.path.join(os.getcwd(), "cpp/engine.dll")
        self.dll = ctypes.CDLL(path)
        self._setup_functions()


    def _setup_functions(self):
        # Set up game functions
        self.dll._Z10init_statev.restype = ctypes.c_uint64*32;
        
        self.dll._Z10next_statePmm.argtypes = [ctypes.c_uint64*32];
        self.dll._Z10next_statePmm.restype = ctypes.c_uint64*5;
        
        self.dll._Z10getActionsPm.argtypes = [ctypes.c_uint64*32];
        self.dll._Z10getActionsPm.restype = ctypes.c_uint64*512;
        
        self.dll._Z11checkStatusPm.argtypes = [ctypes.c_uint64*32];
        self.dll._Z11checkStatusPm.restype = ctypes.c_ushort;
        
    def init_state(self):
        return self.dll._Z10init_statev()
    
    def next_state(self,state,action):
        return self.dll._Z10next_statePmm(state,action)
    
    def getActions(self,state):
        return self.dll._Z10next_statePmm(state)
    
    def checkStatus(self,state):
        return self.dll._Z11checkStatusPm(state)

NameError: name 'game_rule' is not defined

In [ ]:
a=game_rule_cpp_imp()

In [ ]:
a.init_state()

In [1]:
from game_rule import DLLGameRule 
GR=DLLGameRule()
import time
import random
import ctypes
import math
import time

In [2]:
from game_rule import DLLGameRule 
GR=DLLGameRule()
import time
import random
import ctypes
import math
import time

In [3]:
import random

# contatore globale di valutazioni statiche
evN = 0

def minimax(state, depth, maximizing_player, w, alpha=-1e9, beta=+1e9, currentTurn=0):
    """
    Restituisce (valore, migliore_azione)
    """
    global evN

    # 1) verifica immediata di terminale
    status = GR.checkStatus(state)
    if status != 1:  # 1 == ongoing
        t = currentTurn% 2 == 0
        # se è vittoria per chi muove in questo stato
        if (status == 3 and t == 0) or (status == 4 and t == 1):
            return +1000, None
        else:
            return -1000, None

    # 2) cutoff di profondità → valutazione statica
    if depth == 0:
        evN += 1
        return GR.calcVal(state, w), None

    actionsVect = GR.getActions(state)  # es. [n, a1, a2, ..., a_n]
    n = actionsVect[0]
    localActions=[actionsVect[i] for i in range(1,n+1)]
    best_action = localActions[0]
    
    if maximizing_player:
        max_eval = -1e9
        for action in localActions:
            child = GR.copyBoard(state)
            GR.next_state(child, action)
            evalV, _ = minimax(child, depth-1, False, w, alpha, beta,currentTurn=currentTurn+1)
            GR.delBoard(child)

            if evalV > max_eval:
                max_eval = evalV
                best_action = action

            alpha = max(alpha, evalV)
            if beta <= alpha:
                break  # β-cut

        return max_eval, best_action

    else:
        min_eval = +1e9
        for action in localActions:
            child = GR.copyBoard(state)
            GR.next_state(child, action)
            evalV, _ = minimax(child, depth-1, True, w, alpha, beta,currentTurn=currentTurn+1)
            GR.delBoard(child)

            if evalV < min_eval:
                min_eval = evalV
                best_action = action

            beta = min(beta, evalV)
            if beta <= alpha:
                break  # α-cut

        return min_eval, best_action


def main():
    Win = [0, 0]
    
    w=(ctypes.c_double * 6)(0,0,0,1,-2,30)

    for Q in range(100):
        state = GR.init_state()
        i = 0
        totMov = 0
        while i < 300:
            global evN
            if i % 2 == 0:
                evN = 0
                score, best_action = minimax(state, 5, True, w,currentTurn=i+1)
                print(f"Score:: {score}")
            else:
                actions = GR.getActions(state)
                # scelta casuale per il secondo giocatore
                m = random.randint(1, actions[0])
                best_action = actions[m]

            print(f"Best action with: {evN} iteration for turn {i}")
            GR.next_state(state, best_action)
            totMov += 1
            i += 1

            status = GR.checkStatus(state)
            if status != 1:
                Win[i % 2] += 1
                break
        GR.delBoard(state)
        print(f"Win: {Win[0]}   {Win[1]}")

In [ ]:
main()

In [ ]:

state = GR.init_state()


In [ ]:
V=GR.toVect(state)

In [ ]:
act=GR.getActions(state)

In [ ]:
GR.actionToString(act[4])

In [ ]:
";".join([GR.actionToString(act[i]) ])

In [1]:
";".join([GR.actionToString(act[i]).decode() for i in range(1,act[0]+1)])

NameError: name 'act' is not defined

In [1]:
from subprocess import Popen, PIPE
file_exe="../../../Mzinga.LinuxX64/MzingaEngine"
player = Popen(file_exe, stdin=PIPE, stdout=PIPE, stderr=PIPE, encoding="UTF8", bufsize=1)

player.stdout.readline()
player.stdout.readline()
player.stdout.readline()
from game_rule import DLLGameRule 
GR=DLLGameRule()
import time
import random
import ctypes
import math
import time



In [2]:

interactionProgram=""
def initNewGame(player,gamemode=0):
    global interactionProgram
    b=""
    match gamemode:
        case 0:
            b="Base"
        case 1:
            b="Base+M"
        case 2:
            b="Base+L"
        case 3:
            b="Base+P"
        case 4:
            b="Base+ML"
        case 5:
            b="Base+MP"
        case 6:
            b="Base+LP"
        case 7:
            b="Base+MLP"
            
        
    player.stdin.write("newgame "+b+"\n")
    interactionProgram+=" > newgame\n"
    
    interactionProgram+=" < "+player.stdout.readline()+"\n"
    interactionProgram+=" < "+player.stdout.readline()+"\n"
    
def getValidMoves(player):
    global interactionProgram
    player.stdin.write("validmoves\n")
    interactionProgram+=" > validmoves\n"
    read=player.stdout.readline()
    s=read[:-1].split(";")  
    interactionProgram+=" < "+read+"\n"
    interactionProgram+=" < "+player.stdout.readline()+"\n"
    return s

def playMove(p):
    global interactionProgram
    global risAct
    player.stdin.write("play "+p+"\n")
    interactionProgram+=" > play "+p+"\n"
    risAct=player.stdout.readline()
    interactionProgram+=" < "+risAct+"\n"
    interactionProgram+=" < "+player.stdout.readline()+"\n"
    
def playBoth(p,state,act):
    playMove(act)
    GR.next_state(state,GR.stringToAction(state,act))
    
    

In [3]:
initNewGame(player,1)
state = GR.init_state(1)

#GR.next_state(state,9)

#[GR.boardToCVect_low(state)[i] for i in range(284)]


GT:1


In [4]:

actTMPC = GR.getActions(state)
actTMPC[0]


5

In [5]:
[GR.actionToString(actTMPC[i]) for i in range(1,actTMPC[0]+1)]

['wS1', 'wB1', 'wG1', 'wA1', 'wM']

In [6]:
getValidMoves(player)

['wS1', 'wB1', 'wG1', 'wA1', 'wM']

In [7]:
actTMPC[0]

5

In [8]:
evN=0
v1=[]
v2=[]
v2r=[]
moves="";
allVe=[]

def test():
    global v1
    global v2
    global v2r
    global allVe
    
    state = None
    for Q in range(100):
        
        geConf=""
        state = GR.init_state(3)
        initNewGame(player,3)

        iM = 0
        totMov = 0
        while iM <1000:
            global evN

            actTMPC = GR.getActions(state)
            actStrTE = getValidMoves(player)

            actNumE=[GR.stringToAction(state,mov) for mov in actStrTE]
            actNumC=[actTMPC[i] for i in range(1,actTMPC[0]+1)]
            
            actStrC=[GR.actionToString(x) for x in actNumC]
            actStrE=[GR.actionToString(x) for x in actNumE]
            actNumE.sort()
            actNumC.sort()
            actStrE.sort()
            actStrC.sort()

            
            lE=len(actNumE)
            lC=len(actNumC)
            
            if lE!=lC:
                print("ERROR: Different lenght")
                v1=[GR.actionToString(x) for x in actNumE]
                v2=[GR.actionToString(x) for x in actNumC]
                v2r=actNumC
                allVe=(actStrTE,actTMPC,actNumE,actNumC,actStrE,actStrC)
                print(geConf)
                return
            else:
                for i in range(lE):
                    if(actStrC[i]!=actStrE[i]):
                        print("ERROR : not same actions")
                        v1=[GR.actionToString(x) for x in actNumE]
                        v2=[GR.actionToString(x) for x in actNumC]
                        allVe=(actTMPC,actStrTE,actNumE,actNumC,actStrE,actStrC)
                        v2r=actNumC
                        print(geConf)
                        return
            # scelta casuale per il secondo giocatore
            
            m = random.randint(1, lC)

            geConf+=(f"{iM+1}. {actStrC[m-1]}\n")
            playBoth(player,state,actStrC[m-1])
            
            #GR.next_state(state,GR.stringToAction(state,actStrC[m-1]))
    
            totMov += 1
            iM += 1
            

            status = GR.checkStatus(state)
            if status != 1:
                print("iM",iM)
                iM=1000
                break
        print("Q:",Q)
        GR.delBoard(state)
test()

GT:3
ERROR: Different lenght
1. wA1
2. bS1 wA1-
3. wQ -wA1
4. bQ bS1/
5. wP /wQ
6. bP bQ\
7. wS1 wQ/
8. bS2 bQ-
9. wS2 \wP
10. bQ bS2\
11. wA2 /wA1
12. bA1 bQ\
13. wA3 /wS2
14. bG1 bA1\
15. wA3 wS1/
16. bB1 bG1-
17. wA3 \wS2
18. bG2 bB1-
19. wA2 /bB1
20. bB2 \bS2
21. wB1 /wA2
22. bB2 bS2/
23. wG1 wB1\
24. bG3 bB2-
25. wA3 /bS1
26. bA2 bQ-
27. wS1 \bS2
28. bA2 -wS1
29. wA3 wA2\
30. bA3 \bA2
31. wG1 /bG2
32. bA3 /bA2
33. wA3 /bQ



In [11]:
l1=len(v1)
l2=len(v2)
l1,l2
#[(allVe[2][i]==allVe[0][i+1],allVe[2][i],allVe[0][i+1]) for i in range(l1)]

(84, 86)

In [12]:
(l1,l2)

(84, 86)

In [13]:
v1.sort()
v2.sort()

In [14]:
for i in range(max(l1,l2)):
    if((v1[i] if i<l1 else "       ")!=(v2[i] if i<l2 else 0)):
        print(v1[i] if i<l1 else "       ","\t",v2[i] if i<l2 else 0)

        	 wA3 /bP
        	 wA3 /wS1


In [36]:
v2

['bA1 -bQ',
 'bA1 /bQ',
 'bA1 /bS1',
 'bA1 /bS1',
 'bA1 bP-',
 'bA1 bP\\',
 'bA1 bP\\',
 'bA1 bQ\\',
 'bA1 bS1-',
 'bB1 -bQ',
 'bB1 /bQ',
 'bB1 /bS1',
 'bB1 bP-',
 'bB1 bP\\',
 'bB1 bQ\\',
 'bB1 bS1-',
 'bG1 -bQ',
 'bG1 /bQ',
 'bG1 /bS1',
 'bG1 bP-',
 'bG1 bP\\',
 'bG1 bQ\\',
 'bG1 bS1-',
 'bQ bP-',
 'bQ bS1-',
 'bS2 -bQ',
 'bS2 /bQ',
 'bS2 /bS1',
 'bS2 bP-',
 'bS2 bP\\',
 'bS2 bQ\\',
 'bS2 bS1-']

In [17]:
9227321486950268931>>56

128

In [13]:
v2

['wA2 -wB1',
 'wA2 /wB1',
 'wA2 \\wA1',
 'wA2 \\wB1',
 'wA2 wA1/',
 'wA2 wB1/',
 'wA2 wQ-',
 'wA2 wQ/',
 'wB1 /wP',
 'wB1 /wP',
 'wB1 /wQ',
 'wB1 \\wP',
 'wB1 wP',
 'wB2 -wB1',
 'wB2 /wB1',
 'wB2 \\wA1',
 'wB2 \\wB1',
 'wB2 wA1/',
 'wB2 wB1/',
 'wB2 wQ-',
 'wB2 wQ/',
 'wG1 -wB1',
 'wG1 /wB1',
 'wG1 \\wA1',
 'wG1 \\wB1',
 'wG1 wA1/',
 'wG1 wB1/',
 'wG1 wQ-',
 'wG1 wQ/',
 'wQ wA1/',
 'wQ wA1\\',
 'wS1 -wB1',
 'wS1 /wB1',
 'wS1 \\wA1',
 'wS1 \\wB1',
 'wS1 wA1/',
 'wS1 wB1/',
 'wS1 wQ-',
 'wS1 wQ/']

In [ ]:
totMov = 0
t_start = time.time()

for rt in range(5):
    initNewGame(player)    
    i = 0

    while i<200000:

        # .EBoard_getNextsActions(board, ris)
        actStrE = getValidMoves(player)

        num_actions = len(actStrE)
        m = random.randint(1, num_actions)
        if num_actions<=1:
            print(i)
            break
        playMove(actStrE[m-1])
        
        totMov += 1
        
        i += 1

    
t_end = time.time()
print("Tempo totale mossa", (t_end - t_start) /totMov, "s")
print(totMov)


In [16]:
(9227321486950268931>>56)&31


0